In this notebook we implement a simple information retrieval system using inverted index for indexer and we will return the documents using a ranking algoritm based of number of occurences the term has appeared in the documents

# Indexing system

In [1]:
#!pip install nltk
#!pip install pandas

In [2]:
import nltk
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/tsakalos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tsakalos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tsakalos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tsakalos/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
def preprocess_doc(text: str) -> list:

  # clean text
  text = text.replace("\n", " ")
  text = text.strip()
  text = re.sub(r'[^\w]', ' ', text)

  # normalisation, case-folding
  text = text.lower()

  # tokenise
  word_tokens = word_tokenize(text)

  # remove stop words
  stop_words = set(stopwords.words('english'))
  filtered_word_tokens = [w for w in word_tokens if not w in stop_words]

  # retrieve stem from words
  ps = PorterStemmer()
  stemming = []

  for w in filtered_word_tokens:
    stem = ps.stem(w)
    stemming.append(stem)
  
  # retrieve lemma from words
  wordnet_lemmatizer = WordNetLemmatizer()

  lemmatization = []
  for w in stemming:
    lemma = wordnet_lemmatizer.lemmatize(w)
    lemmatization.append(lemma)

  return lemmatization


In [4]:
def create_inverted_index(documents: dict, inverted_index: dict) -> dict:
    
    for id, doc in documents.items():

      doc = preprocess_doc(doc)

      for w in doc:
        if w in inverted_index:
          if id in inverted_index[w]:
            inverted_index[w][id] += 1
          else:
            inverted_index[w][id] = 1
        else:
          inverted_index[w] = {id: 1}

    return inverted_index


In [5]:
documents = {
    "Doc 1": "This is a document about information retrieval.",
    "Doc 2": "This is a document about medium topics related to data.",
    "Doc 3": "This is an article related to mediocre tables.",
    "Doc 4": "Completely irrelevant document.",
    "Doc 5": "Pythagoras of Samos[a] (Ancient Greek: Πυθαγόρας ὁ Σάμιος, romanized: Pythagóras ho Sámios, lit. 'Pythagoras the Samian', or simply Πυθαγόρας; Πυθαγόρης in Ionian Greek; c. 570 – c. 495 BC)[b] was an ancient Ionian Greek philosopher and the eponymous founder of Pythagoreanism. His political and religious teachings were well known in Magna Graecia and influenced the philosophies of Plato, Aristotle, and, through them, the West in general. Knowledge of his life is clouded by legend, but he appears to have been the son of Mnesarchus, a gem-engraver on the island of Samos. Modern scholars disagree regarding Pythagoras's education and influences, but they do agree that, around 530 BC, he travelled to Croton in southern Italy, where he founded a school in which initiates were sworn to secrecy and lived a communal, ascetic lifestyle. This lifestyle entailed a number of dietary prohibitions, traditionally said to have included vegetarianism, although modern scholars doubt that he ever advocated complete vegetarianism.",
    "Doc 6": "Leonardo di ser Piero da Vinci[b] (15 April 1452 – 2 May 1519) was an Italian polymath of the High Renaissance who was active as a painter, draughtsman, engineer, scientist, theorist, sculptor, and architect.[3] While his fame initially rested on his achievements as a painter, he also became known for his notebooks, in which he made drawings and notes on a variety of subjects, including anatomy, astronomy, botany, cartography, painting, and paleontology. Leonardo is widely regarded to have been a genius who epitomized the Renaissance humanist ideal,[4] and his collective works comprise a contribution to later generations of artists matched only by that of his younger contemporary, Michelangelo.",
    "Doc 7": "Another document about information retrieval. Information retrieval is very informative. It offers you information.",
    "Doc 8": "Another document about information.",
    "Doc 9": "Arithmetic is the branch of mathematics that deals with the study of numbers using various operations on them, which is taught in elementary school. Algebra is taught in high school which is also branch of mathematics.",
    "Doc 10": "Mathematics is an area of knowledge that includes the topics of numbers, formulas and related structures, shapes and the spaces in which they are contained, and quantities and their changes.",
    "Doc 11": "Geometry is branch of mathematics. Calculus is branch of mathematics. Discrete mathematics is branch of mathematics."
}

In [6]:
def create_sorted_inverted_index(documents: dict) -> dict:
    
  inverted_index = create_inverted_index(documents, dict())
  sort_inverted_index = dict()
  for i in inverted_index.keys():
    sort_inverted_index[i] = sorted(inverted_index[i].items(), key=lambda x:x[0], reverse=True)

  return sort_inverted_index

# Query system

For the query system, let's implement a simple query system which returns the 3 documents that contain the most number of words matched in the query.

In [7]:
def intersect_posting(ii1, ii2):

  a1 = [x[0] for x in ii1]
  a2 = [x[0] for x in ii2]
  answers = []
  i = 0
  j = 0
  while i < len(a1) and j < len(a2):
    if a1[i] == a2[j]:
      answers.append((a1[i], sum([ii1[i][1], ii2[j][1]])))
      i += 1
      j += 1
    elif a1[i] < a2[j]:
      i += 1
    else:
      j += 1
  return answers

In [8]:
def set_frequency(inverted_index, query_list):
  term_freq = dict()
  for q in query_list:
    doc_list = inverted_index.get(q, (None, 0))
    term_freq[q] = sum([x[1] for x in doc_list])
  sorted_terms = sorted(term_freq.items(), key=lambda x:x[1])
  sorted_terms = [x[0] for x in sorted_terms]
  return sorted_terms


In [9]:
def rest(terms):
  try:
    terms = terms[1:]
  except:
    terms = None
  return terms

In [10]:

def intersect(query_list, sort_inverted_index):
  terms = set_frequency(sort_inverted_index, query_list)
  first_term = terms[0]
  result = sort_inverted_index[first_term]
  # rest of terms
  terms = rest(terms)
  while len(terms) > 0 and len(terms) > 0:
    first_term = terms[0]
    terms = rest(terms)
    result = intersect_posting(result, sort_inverted_index[first_term])
  return result

# Evaluation metrics


In [11]:
def construct_eval_df(query_list: list, documents: dict, labels: tuple):
  inverted_index = create_sorted_inverted_index(documents)
  results = pd.DataFrame([], columns = ["Documents", "Frequency", "Query", "rank"])

  for q in query_list:
    query = preprocess_doc(q)
    temp_results = intersect(query, inverted_index)
    temp_results = pd.DataFrame(temp_results, columns=["Documents", "Frequency"])
    temp_results["Query"] = q
    temp_results["rank"] = temp_results["Frequency"].rank(method='dense', ascending=False)
    results = pd.concat((results, temp_results), axis=0)

  labels = pd.DataFrame(labels, columns=["Query", "Documents"])
  labels["IsRelevant"] = 1

  results = results.merge(labels, how="left")
  results["IsRelevant"].fillna(0, inplace=True)
  return results

In [12]:
labels = (
    ("Engineer or Scientist", ""),
    ("Information RETrieval", "Doc 7"),
    ("Mathematics", "Doc 10")
)

In [13]:
query_list = ["Engineer or Scientist", "Information RETrieval", "Mathematics"]

In [14]:
preprocess_doc("Information RETrieval")

['inform', 'retriev']

In [15]:
eval_df = construct_eval_df(query_list, documents, labels)

In [16]:
eval_df

,Documents,Frequency,Query,rank,IsRelevant
0,Doc 6,2,Engineer or Scientist,1.0,0.0
1,Doc 9,2,Mathematics,2.0,0.0
2,Doc 11,4,Mathematics,1.0,0.0
3,Doc 10,1,Mathematics,3.0,1.0


## Mean reciprocal rank

In [17]:
def mean_reciprocal_rank(df):

  # we have only one or no document matching the information need
  df["reciprocal_rank"] = df["IsRelevant"] / df["rank"]
  
  mean_rr = df.groupby(["Query"])["reciprocal_rank"].max().mean()

  return mean_rr


In [18]:
mean_reciprocal_rank(eval_df)

0.16666666666666666

## Mean average precision


In [19]:
def mean_average_precision(df):
  
  for q in df["Query"].unique():
    ranks = sorted(df[df["Query"]==q]["rank"].unique())
    query_precision = list()
    for r in ranks:
      precision = df[(df["Query"]==q)&(df["rank"]>=r)]["IsRelevant"].max() / r
      query_precision.append(precision)
    query_precision.append(sum(query_precision)/len(query_precision))
  map = sum(query_precision) / len(query_precision)
  return map
      

In [20]:
mean_average_precision(eval_df)

0.611111111111111

# Logging retriever

In [21]:
logs = open("data/app.log")
sentence = dict()
for i, l in enumerate(logs):
  sentence[i] = l


In [2]:
height = 3
buffer = 1
small_bedroom = 2.2* height
big_bathroom = 1.5 * height
living_room = (0.6 + 2.85 + 0.6 + 2.8 + buffer) * height
bedroom = (5 + 1.5 + 3 + 1.5) * height

total = small_bedroom + big_bathroom + living_room + bedroom
total * 2

135.3

156

In [ ]:


inverted_index = create_sorted_inverted_index(documents)
results = pd.DataFrame([], columns = ["Documents", "Frequency", "Query", "rank"])

for q in query_list:
  query = preprocess_doc(q)
  temp_results = intersect(query, inverted_index)
  temp_results = pd.DataFrame(temp_results, columns=["Documents", "Frequency"])
  temp_results["Query"] = q
  temp_results["rank"] = temp_results["Frequency"].rank(method='dense', ascending=False)
  results = pd.concat((results, temp_results), axis=0)

labels = pd.DataFrame(labels, columns=["Query", "Documents"])
labels["IsRelevant"] = 1
